In [1]:
import sys
sys.path.append('..')  # allow this notebook to find equal-level directories
%pylab inline
from importing_modules import *
# pyomo.environ as oe, seaborn as sns, plotly.plotly as py, plotly.graph_objs as go
# from util.gjh_wrapper import gjh_solve, make_df, from vis import acres_bars, zL_bars

Populating the interactive namespace from numpy and matplotlib


In [2]:
from sys import path as pylib #im naming it as pylib so that we won't get confused between os.path and sys.path 
# pylib += [os.path.abspath(os.path.join(ROOT_DIR, '../castjeeves'))]
pylib.append(os.path.abspath('/Users/Danny/Desktop/CATEGORIES/CAREER_MANAGEMENT/CRC_ResearchScientist_Optimization/Optimization_Tool/2_ExperimentFolder/CastJeeves'))
# print(pylib)
from CastJeeves.jeeves import Jeeves

In [ ]:
cj = Jeeves()
# print(cj.geo.all_geotypes())

## Create problem instance

In [3]:
# Load data for each set, parameter, etc. to define a problem instance
objwrapper = LoadObj()
# lrsegs = ['N42071SL2_2410_2700']
lrsegs = ['N51133RL0_6450_0000']
data = objwrapper.load_data(savedata2file=False, lrsegs_list=lrsegs)

# Set the cost bound ----
data.totalcostupperbound = 100000
costboundstr = str(round(data.totalcostupperbound, 1))

# Create concrete problem instance using the separately defined optimization model
mdl = objwrapper.create_concrete(data=data)

# Retain only the Nitrogen load objective, and deactivate the others
mdl.PercentReduction['P'].deactivate()
mdl.PercentReduction['S'].deactivate()

# ---- Solver name ----
localsolver = True
solvername = 'ipopt'
# solvername = 'minos'

## Solve problem instance

In [4]:
looptimestamp = datetime.now().strftime('%Y-%m-%d_%H%M%S')

myobj = SolveAndParse(instance=mdl, data=data, localsolver=localsolver, solvername=solvername)

# set filepath for saving information about all of the solver iterates
output_file_name=os.path.join(projectpath, ''.join(['output/single_LoadObj_', looptimestamp, '.iters']))
myobj.modify_ipopt_options(optionsfilepath='ipopt.opt', newoutputfilepath=output_file_name)

merged_df = myobj.solve()
print('\nObjective is: %d' % oe.value(mdl.PercentReduction['N']))


Solver log file: 'logfile_loadobjective.log'
Solver solution file: '/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpqjqlyyfx.pyomo.sol'
Solver problem files: ('/var/folders/qx/2v7d4gls4d10zzr_ptplrx240000gr/T/tmpqjqlyyfx.pyomo.nl',)
Ipopt 3.12.8: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.8, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:        0
Number of nonzeros in inequality constraint Jacobian.:     2628
Number of nonzeros in Lagrangian Hessian.............:     5278

Total number of variable

In [11]:
getattr(mdl, 'lambda').pprint()

lambda : Direction=Suffix.IMPORT, Datatype=Suffix.FLOAT
    Key : Value


In [7]:
temp_df = pd.DataFrame([[k.index(), mdl.dual[k]]
                            for k in mdl.dual.keys()],
                           columns=['key', 'value'])
# temp_df.dropna(inplace=True)
display(temp_df.tail(5))
print(temp_df.shape[0])

# [mdl.BMPS[temp_df.key[x][0]] for x in range(1,temp_df.shape[0])]
# mdl.BMPS[temp_df.key[x][0] for x in temp_df.shape[0]]

temp_df = pd.DataFrame([[k.index(), getattr(mdl, 'lambda')[k]]
                            for k in getattr(mdl, 'lambda').keys()],
                           columns=['key', 'value'])
# temp_df.dropna(inplace=True)
display(temp_df.tail(5))
print(temp_df.shape[0])

,key,value
1103,"(115, N51133RL0_6450_0000, soy)",1.510736e-12
1104,"(115, N51133RL0_6450_0000, swm)",5.748889e-10
1105,"(115, N51133RL0_6450_0000, wat)",1.753441e-12
1106,"(115, N51133RL0_6450_0000, wfp)",3.072721e-12
1107,"(115, N51133RL0_6450_0000, wto)",3.044986e-12


1108


,key,value


0


In [ ]:
from util.solution_wrangler import get_dual_df
dual_df = get_dual_df(mdl)
display(dual_df.tail(5))

In [ ]:
# display(dual_df.loc[dual_df.bmpshortname == 'HRTill'])
# display(dual_df.tail(20))

In [ ]:
# tempdf_withduals = merged_df.merge(dual_df, how='left',
#                                    on=['bmpshortname', 'landriversegment', 'loadsource'])
# display(tempdf_withduals.head(30))

## Visualize

In [ ]:
dual_df[(dual_df['bmpshortname']=='ConPlan')]
# dual_df.shape
# merged_df.shape
display(dual_df.head(10))
display(dual_df.bmpshortname.unique())

In [ ]:
# ---- Make Dual Figure ----
# # zL_df_filtered = merged_df.loc[abs(merged_df['zL'])>0.45,:].copy()

dual_df_nonzeros = merged_df.merge(dual_df, how='left',
                       on=['bmpshortname', 'landriversegment', 'loadsource'],
                       sort=False)

keystrs = [str([x, y]) for x, y in zip(dual_df_nonzeros['bmpshortname'], dual_df_nonzeros['loadsource'])]
# Make Figure
fig = plt.figure(figsize=(10, 4))
rects = plt.barh(y=keystrs, width=dual_df_nonzeros['dual'])
ax = plt.gca()

ax.set_position([0.3,0.1,0.5,0.8])

In [ ]:
# ---- Make zL Figure ----
# # zL_df_filtered = merged_df.loc[abs(merged_df['zL'])>0.45,:].copy()
# keystrs = [str([x, y]) for x, y in zip(merged_df['bmpshortname'], merged_df['loadsource'])]
# # Make Figure
# fig = plt.figure(figsize=(10, 4))
# rects = plt.barh(y=keystrs, width=merged_df['zL'])
# ax = plt.gca()

# ax.set_position([0.3,0.1,0.5,0.8])


# filenamestr = ''.join(['output/loadobj_zL_costbound', costboundstr, '_', solvername, '_',
#                        datetime.now().strftime('%Y-%m-%d_%H%M%S'),
#                        '.png'])
# plt.savefig(os.path.join(projectpath, filenamestr))

# ---- Make zL Figure ----
filenamestr = ''.join(['output/loadobj_zL_costbound', costboundstr, '_', solvername, '_',
                       datetime.now().strftime('%Y-%m-%d_%H%M%S'),
                       '.png'])
savefilepathandname = os.path.join(projectpath, filenamestr)

zL_bars(df=merged_df, instance=mdl,
        savefig=True, savefilepathandname=savefilepathandname)

In [ ]:
# ---- Acres Figure ----
sorteddf_byacres = merged_df.sort_values(by='acres')

filenamestr = ''.join(['output/loadobj_x_costbound', costboundstr, '_', solvername, '_',
                       datetime.now().strftime('%Y-%m-%d_%H%M%S'), '.png'])
savefilepathandname = os.path.join(projectpath, filenamestr)

objstr = ''.join(['Objective is: ', str(round(oe.value(mdl.PercentReduction['N']),2))])
coststr = ''.join(['Total cost is: ', str(round(oe.value(mdl.Total_Cost.body),1))])
titlestr = '\n'.join([objstr, coststr, 'labels are (cost per unit, total bmp instance cost)'])

acres_bars(df=sorteddf_byacres, instance=mdl, titlestr=titlestr,
           savefig=True, savefilepathandname=savefilepathandname)

## Gradient, Jacobian, Hessian

In [ ]:
gjh_filename, g = gjh_solve(instance=mdl,
                            keepfiles=True,
                            amplenv=ampl,
                            basegjhpath=os.getcwd())

g_df = make_df(instance=mdl, filterbydf=merged_df, g=g)

g_df = sorteddf_byacres.merge(g_df, how='left',
                              on=['bmpshortname', 'landriversegment', 'loadsource'],
                              sort=False)

In [ ]:
# ---- Make gradient Figure ----
# g_df_filtered = g_df.loc[abs(g_df['g'])>0.001,:].copy()
g_df_filtered = g_df
# sorteddf = g_df_filtered.sort_values(by='g')
# sorteddf_byacres

# sorteddf_byacres
# g_df = g_df.merge(sorteddf_byacres, how='right',
#                on=['bmpshortname', 'landriversegment', 'loadsource'])

keystrs = [str([x, y]) for x, y in zip(g_df_filtered['bmpshortname'], g_df_filtered['loadsource'])]
# Make Figure
fig = plt.figure(figsize=(10, 4))
rects = plt.barh(y=keystrs, width=g_df_filtered['g'])
ax = plt.gca()

ax.set_position([0.3,0.1,0.5,0.8])


filenamestr = ''.join(['output/loadobj_g_costbound', costboundstr, '_', solvername, '_',
                       datetime.now().strftime('%Y-%m-%d_%H%M%S'),
                       '.png'])
plt.savefig(os.path.join(projectpath, filenamestr))

## Use the IPOPT derivative test

In [ ]:
solver = SolverFactory(solvername)

# Using the 'ipopt.opt' options file if in this same directory

mdl.jac_g = oe.Suffix(direction=oe.Suffix.IMPORT)
mdl.grad_f = oe.Suffix(direction=oe.Suffix.IMPORT)

results = solver.solve(mdl, tee=True, symbolic_solver_labels=True, keepfiles=True,
                       logfile='logfile_loadobjective_derivativetest.log')

In [ ]:
mdl.grad_f.pprint()

In [ ]:
mdl.x.pprint()